In [2]:
import pandas as pd
import sqlite3
from datetime import datetime as dt

pd.set_option('display.max_columns', None)

In [26]:
df = pd.read_csv("Sample-Superstore.csv", encoding="ISO-8859-1")
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


# Cleaning/Pre-Processing

Remove spaces + hyphens from all labels:

In [27]:
df.columns = df.columns.str.replace(' ', '')
df.columns = df.columns.str.replace('-', '')

Some product IDs have been assigned to multiple product names. This function loops and creates new product IDs for all unique products:

In [28]:
products = df[["ProductID", "ProductName"]].copy()
products.drop_duplicates(inplace=True)

def replace_duped_ids(x):
    id = x.iloc[0]
    name = x.iloc[1]
    next_id = f"{id[:7]}{int(id[7:]) + 1}"
    while len(df[df["ProductID"] == next_id]) != 0:
        next_id = f"{next_id[:7]}{int(next_id[7:]) + 1}"
    df.loc[(df["ProductID"] == id) & (df["ProductName"] == name), "ProductID"] = next_id

result = products[products["ProductID"].duplicated()][["ProductID", "ProductName"]].apply(replace_duped_ids, axis=1)

Some rows are duplicates of the same product + orderID. They always have the same discount value, so this function aggregates matching rows into a single row:

In [29]:
sales = df[["OrderID", "ProductID", "Quantity", "Sales", "Discount", "Profit"]].copy()
sales.drop_duplicates()

def combine_dupes(x):
    order_id = x.iloc[0]
    product_id = x.iloc[1]
    sales_dupes = df[(df["OrderID"] == order_id) & (df["ProductID"] == product_id)]
    df.at[sales_dupes.index[0], "Sales"] = df.iloc[sales_dupes.index[0]]["Sales"] + df.iloc[sales_dupes.index[1]]["Sales"]
    df.at[sales_dupes.index[0], "Profit"] = df.iloc[sales_dupes.index[0]]["Profit"] + df.iloc[sales_dupes.index[1]]["Profit"]
    df.drop(index=sales_dupes.index[1], inplace=True)

result = sales[sales[["OrderID", "ProductID"]].duplicated()][["OrderID", "ProductID"]].apply(combine_dupes, axis=1)

Reformat date strings to be yyyy-mm-dd:

In [30]:
df["OrderDate"] = df["OrderDate"].apply(lambda x: dt.strptime(x, "%m/%d/%Y").strftime("%Y-%m-%d"))
df["ShipDate"] = df["ShipDate"].apply(lambda x: dt.strptime(x, "%m/%d/%Y").strftime("%Y-%m-%d"))

Assign each unique address an incrementing integer ID:

In [ ]:
df["AddressCombi"] = df.apply(lambda x: f"{x["City"]} {x["State"]} {x["Region"]} {x["Country"]} {x["PostalCode"]}", axis=1)
df["AddressID"] = pd.factorize(df["AddressCombi"])[0]
df["AddressID"] = df["AddressID"].apply(lambda x: x + 1)
df.drop(labels="AddressCombi", axis=1, inplace=True)

Create columns for each category + subcategory code that match the productID:

In [8]:
df["CategoryCode"] = df.apply(lambda x: x["Category"].upper()[:3], axis=1)
df["SubCategoryCode"] = df.apply(lambda x: x["SubCategory"].upper()[:2], axis=1)

In [9]:
df.head()

,RowID,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,AddressID,CategoryCode,SubCategoryCode
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,1,FUR,BO
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,1,FUR,CH
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,2,OFF,LA
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,3,FUR,TA
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,3,OFF,ST


# Convert to SQL

In [10]:
conn = sqlite3.connect("superstore+.db")
cur = conn.cursor()

### Customers

In [11]:
# drop_table_customer = "DROP TABLE IF EXISTS Customer"
create_table_customer = """
    CREATE TABLE IF NOT EXISTS Customer (
        CustomerID TEXT PRIMARY KEY,
        CustomerName TEXT,
        Segment TEXT
    );
"""
cur.execute(create_table_customer)

customers = df[["CustomerID", "CustomerName", "Segment"]].copy()
customers.drop_duplicates(inplace=True)
customers.to_sql("Customer", con=conn, index=False, if_exists="append")

793

### Addresses

In [12]:
# drop_table_address = "DROP TABLE IF EXISTS Address;"
create_table_address = """
    CREATE TABLE IF NOT EXISTS Address (
        AddressID INTEGER PRIMARY KEY,
        City TEXT,
        State TEXT,
        Region TEXT,
        Country TEXT,
        PostalCode TEXT
    );
"""
cur.execute(create_table_address)

addresses = df[["AddressID", "City", "State", "Region", "Country", "PostalCode"]].copy()
addresses.drop_duplicates(inplace=True)
addresses.to_sql("Address", con=conn, index=False, if_exists="append")

632

### Address Junctions

In [13]:
# drop_table_addressJunction = "DROP TABLE IF EXISTS AddressJunction;"
create_table_addressJunction = """
    CREATE TABLE IF NOT EXISTS AddressJunction (
        AddressID INTEGER,
        CustomerID TEXT,
        PRIMARY KEY (AddressID, CustomerID),
        FOREIGN KEY (AddressID) REFERENCES Address (AddressID),
        FOREIGN KEY (CustomerID) REFERENCES Customer (CustomerID)
    );
"""
cur.execute(create_table_addressJunction)

addressJunctions = df[["AddressID", "CustomerID"]].copy()
addressJunctions.drop_duplicates(inplace=True)
addressJunctions.to_sql("AddressJunction", con=conn, index=False, if_exists="append")

4910

### Orders

In [14]:
# drop_table_order = "DROP TABLE IF EXISTS CustomerOrder;"
create_table_order = """
    CREATE TABLE IF NOT EXISTS CustomerOrder (
        OrderID TEXT PRIMARY KEY,
        AddressID INTEGER,
        CustomerID TEXT,
        OrderDate TEXT,
        ShipMode TEXT,
        ShipDate TEXT,
        FOREIGN KEY (AddressID, CustomerID) REFERENCES AddressJunction (AddressID, CustomerID)
    );
"""
cur.execute(create_table_order)

orders = df[["OrderID", "AddressID", "CustomerID", "OrderDate", "ShipMode", "ShipDate"]].copy()
orders.drop_duplicates(inplace=True)
orders.to_sql("CustomerOrder", con=conn, index=False, if_exists="append")

5009

### Categories

In [15]:
# drop_table_category = "DROP TABLE IF EXISTS Category;"
create_table_category = """
    CREATE TABLE IF NOT EXISTS Category (
        CategoryCode TEXT PRIMARY KEY,
        Category TEXT
    );
"""
cur.execute(create_table_category)

categories = df[["CategoryCode", "Category"]].copy()
categories.drop_duplicates(inplace=True)
categories.to_sql("Category", con=conn, index=False, if_exists="append")

3

### Sub-Categories

In [16]:
# drop_table_subcategory = "DROP TABLE IF EXISTS SubCategory;"
create_table_subcategory = """
    CREATE TABLE IF NOT EXISTS SubCategory (
        CategoryCode TEXT,
        SubCategoryCode TEXT,
        SubCategory TEXT,
        PRIMARY KEY (CategoryCode, SubCategoryCode),
        FOREIGN KEY (CategoryCode) REFERENCES Category (CategoryCode)
    );
"""
cur.execute(create_table_subcategory)

sub_categories = df[["CategoryCode", "SubCategoryCode", "SubCategory"]].copy()
sub_categories.drop_duplicates(inplace=True)
sub_categories.to_sql("SubCategory", con=conn, index=False, if_exists="append")

17

### Products

In [17]:
# drop_table_product = "DROP TABLE IF EXISTS Product;"
create_table_product = """
    CREATE TABLE IF NOT EXISTS Product (
        ProductID TEXT PRIMARY KEY,
        ProductName TEXT,
        SubCategoryCode TEXT,
        FOREIGN KEY (SubCategoryCode) REFERENCES SubCategory (SubCategoryCode)
    );
"""
cur.execute(create_table_product)

products = df[["ProductID", "ProductName", "SubCategoryCode"]].copy()
products.drop_duplicates(inplace=True)
products.to_sql("Product", con=conn, index=False, if_exists="append")

1894

### Sales

In [18]:
# drop_table_sale = "DROP TABLE IF EXISTS Sale;"
create_table_sale = """
    CREATE TABLE IF NOT EXISTS Sale (
        OrderID TEXT,
        ProductID TEXT,
        Quantity INTEGER,
        Sales REAL,
        Discount REAL,
        Profit REAL,
        PRIMARY KEY (OrderID, ProductID),
        FOREIGN KEY (OrderID) REFERENCES CustomerOrder (OrderID),
        FOREIGN KEY (ProductID) REFERENCES Product (ProductID)
    );
"""
cur.execute(create_table_sale)

sales = df[["OrderID", "ProductID", "Quantity", "Sales", "Discount", "Profit"]].copy()
sales.drop_duplicates()
sales.to_sql("Sale", con=conn, index=False, if_exists="append")

9986

# LMS Queries

In [4]:
conn = sqlite3.connect("superstore+.db")
cur = conn.cursor()

#### 1. What is the category generating the maximum sales revenue?

In [5]:
question1 = """
    SELECT 
        Category.Category,
        SUM(Sale.Sales) AS Total_Revenue 
    FROM Sale 
    JOIN Product ON Sale.ProductID = Product.ProductID
    JOIN SubCategory ON Product.SubCategoryCode = SubCategory.SubCategoryCode
    JOIN Category ON SubCategory.CategoryCode = Category.CategoryCode
    GROUP BY Category.Category
    ORDER BY Total_Revenue DESC;
"""

cur.execute(question1)
question1_result = pd.DataFrame(cur.fetchall(), columns=["Category", "Total_Revenue"])
question1_result

,Category,Total_Revenue
0,Technology,835619.5610
1,Furniture,740740.0193
2,Office Supplies,716498.2760


- What about the profit in this category?

In [6]:
question1_1 = """
    WITH highest_revenue_category AS (
        SELECT CategoryCode FROM (
            SELECT 
                Category.Category,
                Category.CategoryCode,
                SUM(Sale.Sales) AS Total_Revenue 
            FROM Sale 
            JOIN Product ON Sale.ProductID = Product.ProductID
            JOIN SubCategory ON Product.SubCategoryCode = SubCategory.SubCategoryCode
            JOIN Category ON SubCategory.CategoryCode = Category.CategoryCode
            GROUP BY Category.Category
            ORDER BY Total_Revenue DESC
            LIMIT 1
        )
    )

    SELECT SUM(Sale.Profit) AS Total_Profit
    FROM Sale
    JOIN Product ON Sale.ProductID = Product.ProductID
    JOIN SubCategory ON Product.SubCategoryCode = SubCategory.SubCategoryCode
    WHERE SubCategory.CategoryCode = (
        SELECT * FROM highest_revenue_category
    );
"""

cur.execute(question1_1)
question1_1_result = pd.DataFrame(cur.fetchall(), columns=["Total_Profit"])
question1_1_result

,Total_Profit
0,145373.1164


- Are they making a loss in any categories?

In [ ]:
question1_2 = """
    SELECT 
        Category.Category,
        SubCategory.SubCategory,
        SUM(Sale.Profit) AS Total_Profit
    FROM Sale 
    JOIN Product ON Sale.ProductID = Product.ProductID
    JOIN SubCategory ON Product.SubCategoryCode = SubCategory.SubCategoryCode
    JOIN Category ON SubCategory.CategoryCode = Category.CategoryCode
    GROUP BY SubCategory.SubCategory
    HAVING Total_Profit < 0;
"""

cur.execute(question1_2)
question1_2_result = pd.DataFrame(cur.fetchall(), columns=["Category", "SubCategory", "Total_Profit"])
question1_2_result

,Category,SubCategory,Total_Profit
0,Furniture,Bookcases,-3472.5560
1,Office Supplies,Supplies,-1189.0995
2,Furniture,Tables,-17725.4811


#### 2. What are 5 states generating the maximum and minimum sales revenue?

In [ ]:
question2_max = """
    SELECT 
        Address.State, 
        SUM(Sale.Sales) AS Total_Revenue
    FROM Sale
    JOIN CustomerOrder ON Sale.OrderID = CustomerOrder.OrderID
    JOIN Customer ON CustomerOrder.CustomerID = Customer.CustomerID
    JOIN AddressJunction ON Customer.CustomerID = AddressJunction.CustomerID
    JOIN Address ON AddressJunction.AddressID = Address.AddressID
    GROUP BY Address.State
    ORDER BY Total_Revenue DESC
    LIMIT 5;
"""

cur.execute(question2_max)
question2_max_result = pd.DataFrame(cur.fetchall(), columns=["State", "Total_Revenue"])
question2_max_result



## cali, ny, texas, washington, pensil

,State,Total_Revenue
0,California,3.309032e+06
1,New York,1.854211e+06
2,Texas,1.543716e+06
3,Pennsylvania,8.845245e+05
4,Washington,8.604091e+05


In [ ]:
question2_min = """
    SELECT 
        Address.State, 
        SUM(Sale.Sales) AS Total_Revenue
    FROM Sale
    JOIN CustomerOrder ON Sale.OrderID = CustomerOrder.OrderID
    JOIN Customer ON CustomerOrder.CustomerID = Customer.CustomerID
    JOIN AddressJunction ON Customer.CustomerID = AddressJunction.CustomerID
    JOIN Address ON AddressJunction.AddressID = Address.AddressID
    GROUP BY Address.State
    ORDER BY Total_Revenue ASC
    LIMIT 5;
"""

cur.execute(question2_min)
question2_min_result = pd.DataFrame(cur.fetchall(), columns=["State", "Total_Revenue"])
question2_min_result

## nd, wevag, maine, sd, wyoming

,State,Total_Revenue
0,Wyoming,2453.2840
1,North Dakota,6113.2625
2,West Virginia,11405.4790
3,District of Columbia,15420.3060
4,Maine,18239.6990


#### 3. What are the 3 products in each product segment with the highest sales?

In [25]:
question3 = """
    SELECT Segment, ProductName, Total_Sales FROM (
        SELECT *,
            SUM(Sale.Sales) AS Total_Sales,
            RANK() OVER(
                PARTITION BY Customer.Segment
                ORDER BY SUM(Sale.Sales) DESC
            ) AS Ranked 
        FROM Product
        JOIN Sale ON Product.ProductID = Sale.ProductID
        JOIN CustomerOrder ON Sale.OrderID = CustomerOrder.OrderID
        JOIN Customer ON CustomerOrder.CustomerID = Customer.CustomerID
        GROUP BY Sale.ProductID, Customer.Segment
    )
    WHERE Ranked <= 3;
"""

cur.execute(question3)
question3_result = pd.DataFrame(cur.fetchall(), columns=["Segment", "ProductName", "Total_Sales"])
question3_result

,Segment,ProductName,Total_Sales
0,Consumer,Canon imageCLASS 2200 Advanced Copier,32899.906
1,Consumer,HP Designjet T520 Inkjet Large Format Printer ...,18374.895
2,Consumer,HON 5400 Series Task Chairs for Big and Tall,17384.304
3,Corporate,Canon imageCLASS 2200 Advanced Copier,17499.950
4,Corporate,"3D Systems Cube Printer, 2nd Generation, Magenta",14299.890
5,Corporate,Martin Yale Chadless Opener Electric Letter Op...,8328.100
6,Home Office,Cisco TelePresence System EX90 Videoconferenci...,22638.480
7,Home Office,Canon imageCLASS 2200 Advanced Copier,11199.968
8,Home Office,Hewlett Packard LaserJet 3310 Copier,9239.846


- Are they the 3 most profitable products as well?

In [26]:
question3_1 = """
    SELECT Segment, ProductName, Total_Profits FROM (
        SELECT *,
            SUM(Sale.Profit) AS Total_Profits,
            RANK() OVER(
                PARTITION BY Customer.Segment
                ORDER BY SUM(Sale.Profit) DESC
            ) AS Ranked 
        FROM Product
        JOIN Sale ON Product.ProductID = Sale.ProductID
        JOIN CustomerOrder ON Sale.OrderID = CustomerOrder.OrderID
        JOIN Customer ON CustomerOrder.CustomerID = Customer.CustomerID
        GROUP BY Sale.ProductID, Customer.Segment
    )
    WHERE Ranked <= 3;
"""

cur.execute(question3_1)
question3_1_result = pd.DataFrame(cur.fetchall(), columns=["Segment", "ProductName", "Total_Profits"])
question3_1_result

,Segment,ProductName,Total_Profits
0,Consumer,Canon imageCLASS 2200 Advanced Copier,12879.9632
1,Consumer,Ibico EPK-21 Electric Binding System,4630.4755
2,Consumer,HP Designjet T520 Inkjet Large Format Printer ...,4094.9766
3,Corporate,Canon imageCLASS 2200 Advanced Copier,8399.9760
4,Corporate,"3D Systems Cube Printer, 2nd Generation, Magenta",3717.9714
5,Corporate,Fellowes PB300 Plastic Comb Binding Machine,3030.2019
6,Home Office,Hewlett Packard LaserJet 3310 Copier,3935.9344
7,Home Office,Canon imageCLASS 2200 Advanced Copier,3919.9888
8,Home Office,Canon imageCLASS MF7460 Monochrome Digital Las...,1995.9900


#### 4. What are the 3 best-seller products in each product segment? (Quantity-wise)

In [27]:
question4 = """
    SELECT Segment, ProductName, Units_Sold FROM (
        SELECT *,
            SUM(Sale.Quantity) AS Units_Sold,
            RANK() OVER(
                PARTITION BY Customer.Segment
                ORDER BY SUM(Sale.Quantity) DESC
            ) AS Ranked 
        FROM Product
        JOIN Sale ON Product.ProductID = Sale.ProductID
        JOIN CustomerOrder ON Sale.OrderID = CustomerOrder.OrderID
        JOIN Customer ON CustomerOrder.CustomerID = Customer.CustomerID
        GROUP BY Sale.ProductID, Customer.Segment
    )
    WHERE Ranked <= 3;
"""

cur.execute(question4)
question4_result = pd.DataFrame(cur.fetchall(), columns=["Segment", "ProductName", "Units_Sold"])
question4_result

,Segment,ProductName,Units_Sold
0,Consumer,Avery 485,47
1,Consumer,"Angle-D Binders with Locking Rings, Label Holders",40
2,Consumer,GBC Ibimaster 500 Manual ProClick Binding System,38
3,Corporate,Staples,34
4,Corporate,"Global High-Back Leather Tilter, Burgundy",32
5,Corporate,"Acco Pressboard Covers with Storage Hooks, 14 ...",31
6,Home Office,"Global Stack Chair without Arms, Black",31
7,Home Office,Xerox 226,31
8,Home Office,Global Deluxe High-Back Manager's Chair,27


#### 5. What are the top 3 worst-selling products in every category? (Quantity-wise)

In [28]:
question5 = """
    SELECT Category, ProductName, Units_Sold FROM (
        SELECT 
            Product.ProductName,  
            Category.Category,
            SUM(Sale.Quantity) AS Units_Sold,
            RANK() OVER(
                PARTITION BY Category.Category
                ORDER BY SUM(Sale.Quantity)
            ) AS Ranked
        FROM Sale
        JOIN Product ON Sale.ProductID = Product.ProductID
        JOIN SubCategory ON Product.SubCategoryCode = SubCategory.SubCategoryCode
        JOIN Category ON SubCategory.CategoryCode = Category.CategoryCode
        GROUP BY Sale.ProductID, Category.CategoryCode
    )
    WHERE Ranked <=3;
"""

cur.execute(question5)
question5_result = pd.DataFrame(cur.fetchall(), columns=["Category", "ProductName", "Units_Sold"])
question5_result

,Category,ProductName,Units_Sold
0,Furniture,"Bush Saratoga Collection 5-Shelf Bookcase, Han...",1
1,Furniture,Global Enterprise Series Seating Low-Back Swiv...,1
2,Furniture,Ultra Commercial Grade Dual Valve Door Closer,2
3,Furniture,Barricks Non-Folding Utility Table with Steel ...,2
4,Office Supplies,Boston 1900 Electric Pencil Sharpener,1
5,Office Supplies,Xerox 20,1
6,Office Supplies,Eureka Disposable Bags for Sanitaire Vibra Gro...,2
7,Office Supplies,Avery Hi-Liter Pen Style Six-Color Fluorescent...,2
8,Office Supplies,Grip Seal Envelopes,2
9,Office Supplies,Jiffy Padded Mailers with Self-Seal Closure,2


#### 6. How many unique customers per month are there for the year 2016.

In [29]:
question6 = """
    SELECT
        SUBSTR(OrderDate,1,4) AS Year,
        SUBSTR(OrderDate,6,2) AS Month,
        COUNT(DISTINCT CustomerID) AS Counter
    FROM CustomerOrder
    WHERE Year = '2016'
    GROUP BY Month;
"""

cur.execute(question6)
question6_result = pd.DataFrame(cur.fetchall(), columns=["Year", "Month", "Counter"])
question6_result

,Year,Month,Counter
0,2016,01,46
1,2016,02,42
2,2016,03,80
3,2016,04,83
4,2016,05,96
5,2016,06,90
6,2016,07,89
7,2016,08,86
8,2016,09,176
9,2016,10,95


### Join all

In [46]:
join_all = """
    SELECT 
        Customer.CustomerName,
        Customer.CustomerID,
        Customer.Segment,
        Address.City,
        Address.State,
        Address.Region,
        Address.Country,
        Address.PostalCode,
        CustomerOrder.OrderID,
        CustomerOrder.OrderDate,
        CustomerOrder.ShipMode,
        CustomerOrder.ShipDate,
        Category.Category,
        SubCategory.SubCategory,
        Product.ProductID,
        Product.ProductName,
        Sale.Sales,
        Sale.Discount,
        Sale.Profit,
        Sale.Quantity
    FROM Address 
    JOIN AddressJunction ON Address.AddressID = AddressJunction.AddressID
    JOIN Customer ON AddressJunction.CustomerID = Customer.CustomerID
    JOIN CustomerOrder ON AddressJunction.AddressID = CustomerOrder.AddressID
        AND AddressJunction.CustomerID = CustomerOrder.CustomerID
    JOIN Sale ON CustomerOrder.OrderID = Sale.OrderID
    JOIN Product ON Sale.ProductID = Product.ProductID
    JOIN SubCategory ON Product.SubCategoryCode = SubCategory.SubCategoryCode
    JOIN Category ON SubCategory.CategoryCode = Category.CategoryCode;
"""

cur.execute(join_all)
join_all_result = pd.DataFrame(cur.fetchall(), columns=["CustomerName", "CustomerID", "Segment", "City", "State",
                                                        "Region", "Country", "PostalCode", "OrderID", "OrderDate",
                                                        "ShipMode", "ShipDate", "Category", "SubCategory", "ProductID",
                                                        "ProductName", "Sales", "Discount", "Profit", "Quantity"])
join_all_result.to_csv("sqlized_superstore.csv", index=False)

In [18]:
cur.close()
conn.close()